# Load Dependencies

In [1]:
from Script.utils import *

/home/hx-gpu3/anaconda3/envs/feng/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data

In [2]:
data_dir = './Data/data_cn/'
rundata_dir = './Data/rundata_cn/'
df=pd.read_csv(data_dir+"experiment_index.csv", index_col=0)
df = df.sort_values(by='yield')
df.reset_index(drop=True, inplace=True)
df['entry'] = df.index
df.set_index('entry', inplace=True)
ar_ha_smi=df['Aryl_halide_SMILES'].tolist()
add_smi=df['Additive_SMILES'].tolist()
base_smi=df['Base_SMILES'].tolist()
ligand_smi=df['Ligand_SMILES'].tolist()
yield_value=np.array(df['yield'].tolist())

In [3]:
target = 'yield'
defined_chemical_space = {'Aryl_halide_SMILES':sorted(list(set(ar_ha_smi))), 
                          'Additive_SMILES':sorted(list(set(add_smi))), 
                          'Base_SMILES':sorted(list(set(base_smi))),
                          'Ligand_SMILES':sorted(list(set(ligand_smi))), 
                          }

domain=df.drop(['yield'],axis=1)
print('The size of chemical space is: %d'%len(domain))

The size of chemical space is: 3955


# Reaction Optimization

In [4]:
from joblib import Parallel, delayed
random_state = 0
des_name = 'dft' #'ohe','mordred','dft','alldes'
model = 'Ridge'#'DT','ET','GB','KNR','KRR','LSVR','RF','Ridge','SVR','automl','gridsearch',
cc_rate1 = 0.5 # 0-1 exploration proportion of the first Stage
cc_rate2 = 0.2 # 0-1 exploration proportion of the second Stage
run_time = 120 #120s
per_run_time = 30 #30s
task = f'ourwork-{int(cc_rate1*100)}-{int(cc_rate2*100)}-seed' + str(random_state)

desc_map={}
if des_name not in ['ohe','mordred','dft','alldes']:
    print('Error: The selected descriptor is not supported.')
elif des_name == 'alldes':
    desc_map = np.load('./Data/data_cn/all_desc_map.npy',allow_pickle=True).item()
else:
    for i in glob.glob(data_dir+'*.csv'):
        if i.split('/')[-1].split('.')[0].split('_')[-1]==des_name:     
            tem_data=np.array(pd.read_csv(i))
            for j in range(len(tem_data)):
                if des_name in ['ohe','mordred']:
                    desc_map[tem_data[j][0]]=tem_data[j][1:]
                elif des_name=='dft':
                    desc_map[tem_data[j][1]]=tem_data[j][2:]                                   
desc_domain = getdescdomain(domain,desc_map,defined_chemical_space)
norun_seed = []
for seed in range(100):
    if os.path.exists(rundata_dir+f'{task}_{des_name}_{model}_{seed}.npy'):   
        continue  
    else:
        norun_seed.append(seed)
if model in  ['gridsearch','DT','ET','GB','KNR','KRR','LSVR','RF','Ridge','SVR']:
    Parallel(n_jobs=-1)(delayed(yield_optimization_single_line)(seed=seed, domain = domain, desc_domain = desc_domain, model = model, \
                                                                des_name = des_name,random_state=random_state,tem_cc1num=(3955*cc_rate1), \
                                                                tem_cc2num=int(3955*cc_rate2),task = task,run_time = run_time,per_run_time = per_run_time ) for seed in norun_seed) 
elif model == 'automl':
    for seed in norun_seed:
        yield_optimization_single_line(seed=seed, domain = domain, desc_domain = desc_domain, model = model, \
                                                                des_name = des_name,random_state=random_state,tem_cc1num=(3955*cc_rate1), \
                                                                tem_cc2num=int(3955*cc_rate2),task = task,run_time = run_time,per_run_time = per_run_time )   
total_results = get_total_results(rundata_dir+f'results_optimization/{task}_{des_name}_{model}_*.npy',start=0,end=100) 


Run seed: 7
Run seed: 4
Run seed: 8
Run seed: 9
Run seed: 2
Run seed: 5
Run seed: 3
Run seed: 10
Run seed: 1
Run seed: 6
Run seed: 12
Run seed: 11
Run seed: 0
Run seed: 13
Run seed: 14
Run seed: 15
Run seed: 16
Run seed: 17
Run seed: 18
Run seed: 19
Run seed: 20
Run seed: 21
Run seed: 22
Run seed: 23
Run seed: 24
Run seed: 25
Run seed: 26
Run seed: 27
Run seed: 28
Run seed: 29
Run seed: 30
Run seed: 31
Run seed: 32
Run seed: 33
Run seed: 34
Run seed: 35
Run seed: 36
Run seed: 37
Run seed: 38
Run seed: 39
Run seed: 40
Run seed: 41
Run seed: 42
Run seed: 43
Run seed: 44
Run seed: 45
Run seed: 46
Run seed: 47
Run seed: 48
Run seed: 49
Run seed: 50
Run seed: 51
Run seed: 52
Run seed: 53
Run seed: 54
Run seed: 55
Run seed: 56
Run seed: 57
Run seed: 58
Run seed: 59
Run seed: 60
Run seed: 61
Run seed: 62
Run seed: 63
Run seed: 64
Run seed: 65
Run seed: 66
Run seed: 67
Run seed: 68
Run seed: 69
Run seed: 70
Run seed: 71
Run seed: 72
Run seed: 73
Run seed: 74
Run seed: 75
Run seed: 76
Run seed: